In [2]:
import tensorflow as tf
from tensorflow import keras
import pandas as pd
from sklearn.impute import KNNImputer
from sklearn import preprocessing
import matplotlib.pyplot as plt
import numpy as np
import datetime

def obtener_tiempo(dfr):
    #Esta funcion permite obtener una columna Datetime a partir del agno, dia, hora y minuto de los datos iniciales
    dfr['Day'] = pd.to_datetime(dfr['Day'], format='%j').dt.strftime('%m-%d')
    dfr['Hour'] = dfr['Hour'].astype(str).str.zfill(2)
    dfr['Datetime'] = pd.to_datetime(dfr[['Year','Day', 'Hour', 'Minute']]
                   .astype(str).apply(' '.join, 1), format='%Y %m-%d %H %M') 
    return dfr

def sustituir(dfr):
    #Sustitucion de los valores 9999... por NaN (sabemos que la fila 2 son todo valores nulos para cada variable)
    valores_null = dfr.iloc[2, 4:]
    for i in range(len(valores_null)):
        dfr.iloc[:,i+4] = dfr.iloc[:,i+4].replace(valores_null[i], np.nan)
    return dfr

def imputar_por_interpolacion(dfr):
    #Sustituye los valores NaN por valores aproximados mediante el metodo de interpolacion
    dfr = dfr.interpolate(method='linear', limit_direction='forward')
    return dfr

def imputar_por_KNND(dfr):
    #Este metodo es para imputar las filas que no se hayan podido imputar con interpolacion (primera fila)
    imputer = KNNImputer(n_neighbors=3, weights = 'distance')
    dfr[:] = imputer.fit_transform(dfr)
    return dfr

def normalizar_datos(dfr):
    #Dado un DataFrame, devuelve el DataFrame con valores normalizados
    x = dfr.values #returns a numpy array
    standard_scaler = preprocessing.StandardScaler()
    dfr[:] = standard_scaler.fit_transform(x)
    #dfr = pd.DataFrame(x_scaled)
    return dfr

def desnormalizar_datos(dfr):
    #Dado un DataFrame, devuelve el DataFrame con valores desnormalizados
    x = dfr.values
    standard_scaler = preprocessing.StandardScaler()
    dfr[:] = standard_scaler.inverse_transform(x)
    return dfr

def eliminar_gaps(dfr, n):
    #Eliminacion de filas donde alguna columna tiene N NaNs consecutivos
    for columna in range(dfr.shape[1]):
        mask = dfr.iloc[:,columna].notna()
        a = mask.ne(mask.shift()).cumsum()
        dfr = dfr[(a.groupby(a).transform('size') < n) | mask]
    return dfr

def NaN_consecutivos(dfr):
    #Obtencion del numero maximo de NaN consecutivos segun columna
    nans_consecutivos = []
    for columna in range(df.shape[1]):
        nan_columna = max(df.iloc[:,columna].isnull().astype(int).groupby(df.iloc[:,columna].notnull().astype(int).cumsum()).sum())
        nans_consecutivos.append(nan_columna)

    df_nans = pd.DataFrame(columns = ['Variable','Numero de nans consecutivos'])
    df_nans.iloc[:,0] = df.columns
    df_nans.iloc[:,1] = nans_consecutivos
    return df_nans


from tensorflow.keras.layers import *
from tensorflow.keras.models import Sequential
def build_model_complejo(input_timesteps, output_timesteps, num_links, num_inputs):    
    model = Sequential()
    model.add(BatchNormalization(name = 'batch_norm_0', input_shape = (input_timesteps, num_inputs, 1, 1)))
    model.add(ConvLSTM2D(name ='conv_lstm_1',
                         filters = 64, kernel_size = (10, 1),                       
                         padding = 'same', 
                         return_sequences = True))
    
    model.add(Dropout(0.30, name = 'dropout_1'))
    model.add(BatchNormalization(name = 'batch_norm_1'))

    model.add(ConvLSTM2D(name ='conv_lstm_2',
                         filters = 64, kernel_size = (5, 1), 
                         padding='same',
                         return_sequences = False))
    
    model.add(Dropout(0.20, name = 'dropout_2'))
    model.add(BatchNormalization(name = 'batch_norm_2'))
    
    model.add(Flatten())
    model.add(RepeatVector(output_timesteps))
    model.add(Reshape((output_timesteps, num_inputs, 1, 64)))
    
    model.add(ConvLSTM2D(name ='conv_lstm_3',
                         filters = 64, kernel_size = (10, 1), 
                         padding='same',
                         return_sequences = True))
    
    model.add(Dropout(0.20, name = 'dropout_3'))
    model.add(BatchNormalization(name = 'batch_norm_3'))
    
    model.add(ConvLSTM2D(name ='conv_lstm_4',
                         filters = 64, kernel_size = (5, 1), 
                         padding='same',
                         return_sequences = True))
    
    model.add(TimeDistributed(Dense(units=1, name = 'dense_1', activation = 'relu')))
    model.add(Dense(units=1, name = 'dense_2', activation = 'linear'))
    
    optimizer = tf.keras.optimizers.RMSprop(lr=0.004, clipvalue=1.0)
    model.compile(loss = "mse", optimizer = optimizer, metrics = ['mae', 'mse'])
    return model

def baseline(x_val, y_val):
  num_predicciones = len(y_val)
  timesteps_futuros = len(y_val[0])
  timesteps_pasados = len(x_val[0])
  variables = len(x_val[0][0])
  predicciones = np.zeros(shape=(y_val.shape))
  for prediccion in range(num_predicciones):
      dato = x_val[prediccion][timesteps_pasados-1]
      for num_timesteps_a_predecir in range(timesteps_futuros):
          predicciones[prediccion][num_timesteps_a_predecir] = dato
  return predicciones


from sklearn.metrics import r2_score
from sklearn.metrics import mean_absolute_error
from sklearn.metrics import mean_squared_error
def obtener_metricas(y_val, y_preds, nombre_modelos):
    #Aplanamos los datos para que se puedan calcular las metricas
    y_val = np.squeeze(y_val)
    y_val = y_val.ravel()
    datos_mse  = []
    datos_r2   = []
    datos_rmse = []
    datos_mae  = []
    columnas   = []
    for modelo in range(len(nombre_modelos)):
        y_pred = np.squeeze(y_preds[modelo])
        y_pred = y_pred.ravel()
        #Calculamos las metricas
        r2   = r2_score(y_val, y_pred)
        mse  = mean_squared_error(y_val, y_pred)
        mae  = mean_absolute_error(y_val, y_pred)
        rmse = mse**0.5
        #Los anadimos a las listas correspondientes
        datos_mse.append(mse)
        datos_r2.append(r2)
        datos_rmse.append(rmse)
        datos_mae.append(mae)
        #Creamos una columna por modelo
        columnas.append(nombre_modelos[modelo])

    #Mostramos los resultados en forma de df
    nombres_metricas = ['R2', 'RMSE', 'MSE', 'MAE']
    valores = [datos_r2, datos_rmse, datos_mse, datos_mae]

    metricas = pd.DataFrame(valores, columns = columnas, index = nombres_metricas)
    return metricas



def desglose_por_timestep(y_val, predicciones):
    num_predicciones = predicciones.shape[0]
    num_timesteps   = predicciones.shape[1]

    timesteps_y_val  = []
    timesteps_y_pred = []
    
    #Creamos 4 listas, una para cada timestep
    for lista_timestep in range(num_timesteps):
        timestep_valN = []
        timesteps_y_val.append(timestep_valN)
        timestep_predN = []
        timesteps_y_pred.append(timestep_predN)
    #Metemos en cada una de los 4 listas, el correspondiente timestep de cada prediccion
    for prediccion in range(num_predicciones):
        for timestep in range(num_timesteps):
            timesteps_y_pred[timestep].append(predicciones[prediccion][timestep])
            timesteps_y_val[timestep].append(y_val[prediccion][timestep])
    
    return timesteps_y_val, timesteps_y_pred

def comparacion_modelos_timestep(y_val, predicciones, nombre_modelos):
  timesteps_y_preds = []
  for modelo in range(len(nombre_modelos)):
      timesteps_y_val, timesteps_y_pred_modelo = desglose_por_timestep(y_val, predicciones[modelo])
      timesteps_y_preds.append(timesteps_y_pred_modelo)
  for timestep in range(4):
      timesteps_modelos = []
      for modelo in range(len(nombre_modelos)):
          timesteps_modelos.append(timesteps_y_preds[modelo][timestep])
      print("=========TIMESTEP: ",timestep,"=========\n",obtener_metricas(timesteps_y_val[timestep], timesteps_modelos, nombre_modelos))


def comparacion_modelos_timestep_variables(y_val, predicciones, nombre_modelos, nombre_variables):
  desglose_preds = []
  for modelo in range(len(nombre_modelos)):
      desglose_val, desglose_pred = agrupar_variables_timestep(y_val, predicciones[modelo])
      desglose_preds.append(desglose_pred)
  
  for paso in range(4):
    print("===================TIMESTEP ", paso,"===================")
    for variable in range(len(nombre_variables)):
        variableN_modelos = []
        for modelo in range(len(nombre_modelos)):
          variableN_modelos.append(desglose_preds[modelo][paso][variable])
        print("=====VARIABLE", nombre_variables[variable],"====")
        print(obtener_metricas(desglose_val[paso][variable], variableN_modelos, nombre_modelos))


def agrupar_variables_timestep(y_val, y_pred):
    num_predicciones = y_val.shape[0]
    num_timesteps    = y_val.shape[1]
    num_variables    = y_val.shape[2]
    #Creacion de listas
    desglose_val  = []
    desglose_pred = []
    for paso in range(num_timesteps):
        timestepN_val  = []
        timestepN_pred = []
        for i in range(num_variables):
            timestepN_varI_val = []
            timestepN_val.append(timestepN_varI_val)
            
            timestepN_varI_pred = []
            timestepN_pred.append(timestepN_varI_pred)
            
        desglose_val.append(timestepN_val)
        desglose_pred.append(timestepN_pred)
    
    #Desglose
    for prediccion in range(num_predicciones):
        for paso in range(num_timesteps):
            for var in range(num_variables):
                desglose_val[paso][var].append(y_val[prediccion][paso][var])
                desglose_pred[paso][var].append(y_pred[prediccion][paso][var])
    
    return desglose_val, desglose_pred




def resumen_comparativa(y_val, predicciones, nombre_modelos, variables):
    pd.set_option("display.max_rows", None, "display.max_columns", None)

    #1-Comparativa general de los modelos
    print("===================================================COMPARATIVA GENERAL DE LOS MODELOS===================================================")
    print(obtener_metricas(y_val, predicciones, nombre_modelos))

    #2-Comparativa por timestep de los modelos
    print("===================================================COMPARATIVA POR TIMESTEP DE LOS MODELOS===================================================")
    print(comparacion_modelos_timestep(y_val, predicciones, nombre_modelos))

    print("===================================================COMPARATIVA POR TIMESTEP Y VARIABLES DE LOS MODELOS===================================================")
    #3-Comparativa por timestep y variable de los modelos
    print(comparacion_modelos_timestep_variables(y_val, predicciones, nombre_modelos, variables))


def obtener_predicciones_variables(predicciones_tormentas, nombre_variables):
  num_predicciones = len(predicciones_tormentas)
  num_pasos        = len(predicciones_tormentas[0])
  num_variables    = len(predicciones_tormentas[0][0])

  lista_variables = []


  for variable in range(num_variables):
    valores_variable = []
    lista_variables.append(valores_variable)

  for predicciones in range(num_predicciones - 3744, num_predicciones, 4):
    for paso in range(num_pasos):
      for variable in range(num_variables):
        dato = np.squeeze(predicciones_tormentas[predicciones][paso][variable])
        lista_variables[variable].append(dato)

  return lista_variables


def obtener_df_predicciones(predicciones_variables, df_tormenta_real):
    df_predicciones = pd.DataFrame(prediccions_variables)
    df_predicciones = df_predicciones.T
    df_predicciones.columns = variables
    df_predicciones.set_index(df_tormenta_real.index, inplace=True)

    df_predicciones[:] = scaler.inverse_transform(df_predicciones.values)
    df_tormenta_real[:] = scaler.inverse_transform(df_tormenta_real.values)


    nombre_observado = variables+' observado'
    nombre_predicho  = variables+ ' predicho'

    df_predicciones=df_predicciones.astype(float)
    df_tormenta_real=df_tormenta_real.astype(float)

    #Grafico para ver de manera clara donde se encuentran los NaN en cada variable (discontinuidad)
    plt.rcParams.update({'legend.fontsize': 15})
    ax = df_predicciones.plot(subplots = True, figsize=(25, 25), markersize = 20)
    df_tormenta_real.plot(ax = ax, subplots = True, figsize=(25, 20), linestyle = ':', color = 'grey', sharex = False,
                            fontsize=22, markersize = 20)


#1-Montamos Drive para poder acceder a los datos y los leemos

In [3]:
#Leemos los datos
#df = pd.read_csv('datos.csv', header=0)
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [7]:
df = pd.read_csv('/content/drive/MyDrive/TFG_codigo/Prediccion_general/datos.csv', header=0)

#2-Preprocesamos los datos y los normalizamos

In [8]:
df = sustituir(df)
df = imputar_por_interpolacion(df)
imputar_por_KNND(df.iloc[:3, :])
df = obtener_tiempo(df)
df = df.drop(['Year', 'Day', 'Hour', 'Minute'], axis = 1)
df = df.set_index('Datetime')
df

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:33: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
/usr/local/lib/python3.7/dist-packages/pandas/core/indexing.py:1734: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  isetter(loc, value[:, i].tolist())


,IMF(nT),Bx GSM(nT),By GSM(nT),Bz GSM(nT),Flow Speed(km/s),Proton Density(n/cc),Proton Temperature(K)
Datetime,,,,,,,
1995-01-01 00:00:00,1.37,0.130,1.17,-0.670,311.453358,18.422649,17401.558702
1995-01-01 00:05:00,1.26,0.090,1.12,-0.500,311.400000,18.460000,17347.000000
1995-01-01 00:10:00,1.46,0.037,1.30,-0.586,311.560000,18.348000,17510.600000
1995-01-01 00:15:00,1.66,-0.016,1.48,-0.672,311.720000,18.236000,17674.200000
1995-01-01 00:20:00,1.86,-0.069,1.66,-0.758,311.880000,18.124000,17837.800000
...,...,...,...,...,...,...,...
2021-02-18 23:35:00,2.87,2.130,0.03,-1.860,364.100000,4.670000,58514.000000
2021-02-18 23:40:00,2.92,2.210,0.07,-1.880,364.900000,4.400000,62883.000000
2021-02-18 23:45:00,2.90,2.450,0.05,-1.540,364.100000,4.650000,58624.000000


In [9]:
from sklearn.preprocessing import StandardScaler
x = df.values #returns a numpy array
scaler = StandardScaler()
scaler.fit(x)
df[:] = scaler.transform(x)
df

,IMF(nT),Bx GSM(nT),By GSM(nT),Bz GSM(nT),Flow Speed(km/s),Proton Density(n/cc),Proton Temperature(K)
Datetime,,,,,,,
1995-01-01 00:00:00,-1.424353,0.032687,0.291615,-0.189644,-1.146234,2.347966,-0.802707
1995-01-01 00:05:00,-1.460589,0.021451,0.278938,-0.138110,-1.146766,2.355286,-0.803304
1995-01-01 00:10:00,-1.394705,0.006564,0.324574,-0.164180,-1.145172,2.333336,-0.801513
1995-01-01 00:15:00,-1.328821,-0.008324,0.370210,-0.190250,-1.143577,2.311387,-0.799722
1995-01-01 00:20:00,-1.262937,-0.023212,0.415845,-0.216320,-1.141983,2.289437,-0.797931
...,...,...,...,...,...,...,...
2021-02-18 23:35:00,-0.930224,0.594484,0.002589,-0.550381,-0.621617,-0.347237,-0.352641
2021-02-18 23:40:00,-0.913753,0.616956,0.012730,-0.556443,-0.613645,-0.400151,-0.304812
2021-02-18 23:45:00,-0.920341,0.684372,0.007660,-0.453376,-0.621617,-0.351156,-0.351436


#3-Fechas de tormentas geomagnéticas

In [4]:
#ORIGINAL
TRAINING_STORMS_DATES_START = ['19980214', '19980802', '19980919', '19990216', '19991015', '20000709', '20000806', '20000915', '20001101',
                               '20010314', '20010406', '20011017', '20011031', '20020517', '20031115', '20040720', '20050510', '20060409', '20061209', '20120301']
TRAINING_STORMS_DATES_END = ['199802222359', '199808082359', '199809292359', '199902242359', '199910252359', '200007192359', '200008162359', '200009252359', '200011152359',
                             '200103242359', '200104162359', '200110222359', '200111102359', '200205272359', '200311252359', '200407302359', '200505202359', '200604192359', '200612192359', '201203112359']

VALIDATION_STORMS_DATES_START = [
    '19980428', '19990919', '20031025', '20150618', '20170901']
VALIDATION_STORMS_DATES_END = [
    '199805082359', '199909262359', '200311032359', '201506282359', '201709112359']

TESTING_STORMS_DATES_START = ['19980622', '19981102', '19990109', '19990413', '20000116', '20000402', '20000519',
                              '20010326', '20030526', '20030708', '20040118', '20041104', '20120910', '20130528', '20130626', '20150311', '20180822']
TESTING_STORMS_DATES_END = ['199806302359', '199811122359', '199901182359', '199904192359', '200001262359', '200004122359', '200005282359', '200104042359',
                            '200306062359', '200307182359', '200401272359', '200411142359', '201210052359', '201306042359', '201307042359', '201503212359', '201809032359']

#3-Creacion de datasets

In [5]:
def split_tormentas(df, train_start_dates = TRAINING_STORMS_DATES_START, train_end_dates = TRAINING_STORMS_DATES_END,
                    val_start_dates = VALIDATION_STORMS_DATES_START, val_end_dates = VALIDATION_STORMS_DATES_END, test_start_dates = TESTING_STORMS_DATES_START, test_end_dates = TESTING_STORMS_DATES_END):
  
  indice_tormenta = 0
  
  tormentas_train = []
  tormentas_val   = []
  tormentas_test  = []

  for start, end in zip(train_start_dates, train_end_dates):
    tormenta = df.loc[start:end].copy()
    tormenta['tormenta'] = indice_tormenta
    tormentas_train.append(tormenta)
    indice_tormenta = indice_tormenta + 1

  for start, end in zip(val_start_dates, val_end_dates):
    tormenta = df.loc[start:end].copy()
    tormenta['tormenta'] = indice_tormenta
    tormentas_val.append(tormenta)
    indice_tormenta = indice_tormenta + 1

  for start, end in zip(test_start_dates, test_end_dates):
    tormenta = df.loc[start:end].copy()
    tormenta['tormenta'] = indice_tormenta
    tormentas_test.append(tormenta)
    indice_tormenta = indice_tormenta + 1

  df_train = tormentas_train[0]
  df_val   = tormentas_val[0]
  df_test  = tormentas_test[0]

  for tormenta in range(len(tormentas_train)):
    if(tormenta != 0):
      df_train = pd.concat((df_train, tormentas_train[tormenta]))

  for tormenta in range(len(tormentas_val)):
    if(tormenta != 0):
      df_val = pd.concat((df_val, tormentas_val[tormenta]))

  for tormenta in range(len(tormentas_test)):
    if(tormenta != 0):
      df_test = pd.concat((df_test, tormentas_test[tormenta]))
  

  return df_train, df_val, df_test


In [10]:
df_train, df_val, df_test = split_tormentas(df)

df_train.set_index('tormenta', append=True, inplace=True)
df_val.set_index('tormenta', append=True, inplace=True)
df_test.set_index('tormenta', append=True, inplace=True)

df_train = df_train.reorder_levels(['tormenta', 'Datetime'])
df_val = df_val.reorder_levels(['tormenta', 'Datetime'])
df_test = df_test.reorder_levels(['tormenta', 'Datetime'])


df_train

IMF(nT)  ...  Proton Temperature(K)
tormenta Datetime                       ...                       
0        1998-02-14 00:00:00 -0.202208  ...              -0.547556
         1998-02-14 00:05:00 -0.238444  ...              -0.367518
         1998-02-14 00:10:00 -0.258209  ...              -0.356275
         1998-02-14 00:15:00 -0.281268  ...              -0.436890
         1998-02-14 00:20:00 -0.284563  ...              -0.319208
...                                ...  ...                    ...
19       2012-03-11 23:35:00 -0.551392  ...              -0.825987
         2012-03-11 23:40:00 -0.551392  ...              -0.827257
         2012-03-11 23:45:00 -0.584334  ...              -0.777228
         2012-03-11 23:50:00 -0.610687  ...              -0.791646
         2012-03-11 23:55:00 -0.607393  ...              -0.902278

[60768 rows x 7 columns]

In [12]:
def multivariate_multioutput_data(dataset, target, start_index, end_index, history_size,
                      target_size, step, single_step=False):
    data = []
    labels = []

    start_index = start_index + history_size
    if end_index is None:
        end_index = len(dataset) - target_size

    for i in range(start_index, end_index):
        indices = range(i-history_size, i, step)
        data.append(dataset[indices])

        if single_step:
            labels.append(target[i+target_size])
        else:
            labels.append(target[i:i+target_size])

    return np.array(data)[:,:,:,np.newaxis,np.newaxis], np.array(labels)[:,:,:,np.newaxis,np.newaxis]





def timeseries_dataset_from_df1(df, window_entrada = 40, window_salida = 4):
    dataset_x_total = None    
    dataset_y_total = None

    start_ind = 0
    step = 1
    for indice_tormenta, datos_tormenta in df.groupby("tormenta"):
        dataset_x , dataset_y = multivariate_multioutput_data(
            datos_tormenta.values,
            datos_tormenta.values, 
            start_ind,
            None,
            window_entrada,
            window_salida,
            step)

        if dataset_x_total is None:
            dataset_x_total = dataset_x
            dataset_y_total = dataset_y 
        else:
            dataset_x_total = np.concatenate((dataset_x_total, dataset_x))
            dataset_y_total = np.concatenate((dataset_y_total, dataset_y))

    return dataset_x_total, dataset_y_total

In [13]:
#OBTENEMOS LOS datasets x e y
x_train , y_train = timeseries_dataset_from_df1(df_train)
x_val , y_val     = timeseries_dataset_from_df1(df_val)
x_test , y_test   = timeseries_dataset_from_df1(df_test)

In [14]:
x_test.shape, y_test.shape

((53972, 40, 7, 1, 1), (53972, 4, 7, 1, 1))

**CARGAMOS EL MODELO COMPLEJO**

In [15]:
modelo_c = build_model_complejo(40, 4, 7, 7)
modelo_c.load_weights('/content/drive/MyDrive/TFG_codigo/Prediccion_general/Modelo_Complejo/modelo_complejo_150epochs_pesos.h5')

/usr/local/lib/python3.7/dist-packages/tensorflow/python/keras/optimizer_v2/optimizer_v2.py:375: UserWarning: The `lr` argument is deprecated, use `learning_rate` instead.
  "The `lr` argument is deprecated, use `learning_rate` instead.")


Realizamos predicciones

In [16]:
predicciones_tormenta_c = modelo_c.predict(x_test, verbose=1)
predicciones_baseline_tormenta = baseline(x_test, y_test)

1687/1687 [==============================] - 113s 58ms/step


Guardamos las predicciones

In [17]:
np.save('/content/drive/MyDrive/Prediccion_Tormenta/modelo_complejo_predicciones_tormentas_test', predicciones_tormenta_c)
np.save('/content/drive/MyDrive/Prediccion_Tormenta/predicciones_baseline_tormentas_test', predicciones_baseline_tormenta)

Cargamos las predicciones

In [18]:
predicciones_tormenta_c = np.load('/content/drive/MyDrive/Prediccion_Tormenta/modelo_complejo_predicciones_tormentas_test.npy')
predicciones_baseline_tormenta = np.load('/content/drive/MyDrive/Prediccion_Tormenta/predicciones_baseline_tormentas_test.npy')

#6-Evaluación del modelo

Generamos la estructura para que con las funciones podamos obtener las métricas de cada modelo para compararlos

In [19]:
predicciones = [predicciones_tormenta_c, predicciones_baseline_tormenta]
nombre_modelos = ['Complejo', 'Baseline']
variables = df.columns

**Generamos las comparativas entre los modelos**

In [20]:
resumen_comparativa(y_test, predicciones, nombre_modelos, variables)

===================================================COMPARATIVA GENERAL DE LOS MODELOS===================================================
      Complejo  Baseline
R2    0.859157  0.887366
RMSE  0.507406  0.453757
MSE   0.257461  0.205895
MAE   0.231088  0.201620
===================================================COMPARATIVA POR TIMESTEP DE LOS MODELOS===================================================
=========TIMESTEP:  0 =========
       Complejo  Baseline
R2    0.910704  0.951568
RMSE  0.403973  0.297509
MSE   0.163194  0.088512
MAE   0.172211  0.131172
=========TIMESTEP:  1 =========
       Complejo  Baseline
R2    0.869795  0.902237
RMSE  0.487847  0.422725
MSE   0.237995  0.178697
MAE   0.221214  0.190187
=========TIMESTEP:  2 =========
       Complejo  Baseline
R2    0.840469  0.864516
RMSE  0.540046  0.497683
MSE   0.291650  0.247688
MAE   0.253068  0.227749
=========TIMESTEP:  3 =========
       Complejo  Baseline
R2    0.815687  0.831176
RMSE  0.580522  0.555594
MSE   0.337006

**Visualizamos la predicción del modelo complejo**

In [ ]:
df_val = df.iloc[df.shape[0]-549806:]
df_val

,IMF(nT),Bx GSM(nT),By GSM(nT),Bz GSM(nT),Flow Speed(km/s),Proton Density(n/cc),Proton Temperature(K)
Datetime,,,,,,,
2015-11-28 19:10:00,1.154999,1.504596,-1.490711,-1.326419,-1.150752,4.054406,-0.432971
2015-11-28 19:15:00,1.293355,1.504596,-1.620012,-1.390079,-1.129826,3.950538,-0.450771
2015-11-28 19:20:00,1.296649,1.529877,-1.336056,-1.681093,-1.120857,3.746722,-0.427859
2015-11-28 19:25:00,0.983701,1.052349,-1.381692,-1.671999,-1.074022,4.291538,-0.417174
2015-11-28 19:30:00,1.138528,1.277068,-1.394369,-1.380984,-1.084984,3.932900,-0.438127
...,...,...,...,...,...,...,...
2021-02-18 23:35:00,-0.930224,0.594484,0.002589,-0.550381,-0.621617,-0.347237,-0.352641
2021-02-18 23:40:00,-0.913753,0.616956,0.012730,-0.556443,-0.613645,-0.400151,-0.304812
2021-02-18 23:45:00,-0.920341,0.684372,0.007660,-0.453376,-0.621617,-0.351156,-0.351436


In [ ]:
variables = df_val.columns
df_val.columns

Index(['IMF(nT)', 'Bx GSM(nT)', 'By GSM(nT)', 'Bz GSM(nT)', 'Flow Speed(km/s)',
       'Proton Density(n/cc)', 'Proton Temperature(K)'],
      dtype='object')